# Задание:

 *    Выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще);
 *  Научиться извлекать данные из с выбранной веб-страницы;
 *   Научиться генерировать текст по шаблонам.


### Источник  данных и извлечение нужных полей.

В качестве источника данных будем использовать глобальную службу прогнозирования погоды (Global Forecast System (GFS)). Служба предоставляет прогноз погоды на 8 дней с максимальным разрешением в 0.25 градуса. Прогноз распространяется в виде файлов в формате GRIB (GRIdded Binary). Файлы доступны для скачивания:
http://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.2017123006/
Прогноз обновляется каждые 6 часов.
Для работы с grib-файлами нужна библиотека pygrib

Файл прогноза погоды на 6 утра 6 января 2018г, прогноз на 4 часа вперед, то есть на 13:00 по Московскому времени.
 * cd /home/anaconda/var/grib
 * wget http://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.2018010606/gfs.t06z.pgrb2.0p25.f004

In [135]:
import locale
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF8')
# Все даты храняться в временной зоне UTC. Москва +3 часа.
from datetime import timedelta
td = timedelta(hours=3)

In [2]:
import pygrib

In [120]:
filename = '/home/anaconda/var/grib/gfs.t06z.pgrb2.0p25.f004'
myfile = pygrib.open(filename)

In [4]:
# За координаты Москвы примем точку 55.75N,37.75E
# Данные хранятся в двумерных массивах. Индексы - узлы координатной сетки. Широта [90 .. -90], Долгота [0 ... 360 )
# Шаг - четверть градуса
# Идексы в массивах данных для Москвы:
MOSCOW_LAT_INDEX = (90 - 56)*4 + 1 
MOSCOW_LON_INDEX= 37*4  + 3

In [136]:
# Файл содержит GRIB-сообщения. 
# Каждое сообщение содержит значения параметра для всей координатной сетки
# У сообщения есть поля ID и name. 
# Сообщения с одинаковыми именами различаются высотой над уровнем моря, в этом случае select возвращает 
#  массив из нескольких элементов.
# Параметры, которые мы используем, имеют только одну высоту (2 метра или 10 метров), используем только первый эл-т массива.

moscow_t = myfile.select(name = '2 metre temperature')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_p = myfile.select(name = 'Pressure reduced to MSL')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_wg = myfile.select(name = 'Wind speed (gust)')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
# Дата прогноза
# В используемом файле прогноз только на одну дату, возьмем дату из одной из записей.
frcst_date = myfile.select(name = 'Wind speed (gust)')[0].validDate + td

# Осадки
moscow_precip = myfile.select(name = 'Total Precipitation')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]

# Для ветра есть "Восточная"- U и  "Северная"  - V составляющая.
moscow_wu = myfile.select(name = '10 metre U wind component')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_wv = myfile.select(name = '10 metre V wind component')[0].data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
moscow_w = (moscow_wu**2 + moscow_wv**2)**0.5

# Температура указывается в Кельвинах. Переведем в градусы Цельсия.
moscow_t = moscow_t - 273.15
print("Temperature: {:1.2f}; pressure: {:1.2f}, wind: {:1.2f}, gust to {:1.2f}, precip: {:1.2f}".format(
    moscow_t, moscow_p, moscow_w, moscow_wg, moscow_precip))

Temperature: 1.45; pressure: 100942.77, wind: 4.96, gust to 11.00, precip: 0.44


In [137]:
frcst_date 

datetime.datetime(2018, 1, 6, 13, 0)

# Генерация текста по шаблонам

In [8]:
import pymorphy2

from utg import relations as r
from utg import logic
from utg import data
from utg import dictionary
from utg import words
from utg import templates
from utg import constructors

In [138]:
result = get_frcst_date_phrase(frcst_date) + "\n" + \
    get_temp_phrase(moscow_t) + " " + \
    get_wind_phrase(moscow_wv, moscow_wu, moscow_wg) +  " " + \
    get_precipitate_phrase(moscow_precip, moscow_t)
print(result)

Прогноз погоды на 13 часов субботы, 6 января 2018 года.
Ожидается 1 градус тепла. Ветер южный,  5м/с, возможны порывы до 11м/с. Снег с дождем.


Смотрим за окно - и правад, снег с дождем. "Хорошая" рождественская погода :)

# Library

In [99]:
def get_temp_phrase(temp: float):
    degree_word =  words.Word(type=r.WORD_TYPE.NOUN,
                                            forms=['градус', 'градуса', 'градусу', 'градус', 'градусом', 'градусе', 
                                                   'градусы', 'градусов', 'градусам', 'градусы', 'градусами', 'градусах', 
                                                   'градусы', 'градусов', 'градусам', 'градусы', 'градусами', 'градусах'],
                                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))
    warm_word =  words.Word(type=r.WORD_TYPE.NOUN,
                                            forms=['тепло', 'тепла', 'теплу', 'тепло', 'теплом', 'тепле', 
                                                   '', '', '', '', '', '', 
                                                   '', '', '', '', '', ''],
                                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))
    
    frost_word = words.Word(type=r.WORD_TYPE.NOUN,
                            forms=['мороз', 'мороза', 'морозу', 'мороза', 'морозом', 'морозе',  
                                   'морозы', 'морозов', 'морозам', 'морозов', 'морозами', 'морозах', 
                                   'морозы', 'морозов', 'морозам', 'морозов', 'морозами', 'морозах'],
                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))

    #npc = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
    #                            forms=[u'русалка', u'русалки', u'русалке', u'русалку', u'русалкой', u'русалке',
    #                                   u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках',
    #                                   u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках'],
    #                             properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.NEUTER)))
    
    my_dictionary = dictionary.Dictionary(words=[degree_word, warm_word, frost_word])
    template = templates.Template()
    
    if temp > 1:
        template.parse('Ожидается [temp] [градус|temp|вн] [тепло|рд].', externals=('temp'))
        temp = int(round(temp,0))
    elif temp < -1:
        temp = temp * -1
        temp = int(round(temp,0))
        template.parse('Ожидается [temp] [градус|temp|вн] [мороз|рд].', externals=('temp'))
    else:
        return('Ожидается температура около нуля')
        
    result = template.substitute(externals={
                                           'temp': constructors.construct_integer(temp)
                                           }, dictionary=my_dictionary)
    return result
    
get_temp_phrase(-4.4)

'Ожидается 4 градуса мороза.'

In [10]:
def get_wind_direction(V,U):
    # Учтем, что в данных у нас направление ветра (на север), 
    # а в сводках указывают откуда ветер дует (южный).
    
    if U == 0:
        U = 0.001   # Уберем нуль в знаменателе. 
    rel = abs(V/U)
    
    #print(rel)
    if V > 0:
        if rel > 3:
            return("южный")
        elif rel < 0.3:
            if U > 0:
                return("западный")
            else:
                return("восточный")
        else:
            if U > 0:
                return("юго-западный")
            else:
                return("юго-восточный")
    else:
        if rel > 3:
            return("северный")
        elif rel < 0.3:
            if U > 0:
                return("западный")
            else:
                return("восточный")
        else:
            if U > 0:
                return("северо-западный")
            else:
                return("северо-восточный")
get_wind_direction(-100,-10)

'северный'

In [11]:
def get_wind_phrase(V,U, gust):
    direction = get_wind_direction(V,U)
    power = (V*V + U*U)**0.5
    rv = "Ветер {}, {:2.0f}м/с, возможны порывы до {:2.0f}м/с.".format(direction, power, gust)
    return(rv)
get_wind_phrase(3,0,5)

'Ветер южный,  3м/с, возможны порывы до  5м/с.'

In [17]:
def get_precipitate_type(t: int):
    """
    Определим вид осадков по температре t,
    0 - дождь, t > 3
    1 - снег,  t < -1
    2 - снег с дождем
    
    Значения температуры взяты наобум, 
    каких-либо авторитетных источников по этому вопросу найти не удалось.
    TODO: можно учитывать температуру на разных высотах,
    чтобы более точно определять вид осадков.
    Также можно определять вероятность ледяного дождя.
    """
    rv = '';
    if t > 5:
        return 'дождь'
    elif t < - 1:
        return 'снег'
    return 'снег с дождем'
get_precipitate_type(1)

'снег с дождем'

In [18]:
def get_precipitate_phrase(p: int, t: int):
    """
    Размерность предсказаний - мм/ч, 
    """
    rv = ""
    tp = get_precipitate_type(t)
    if p == 0:
        rv = "Осадков не ожидается."
    elif p < 0.05:       
        rv = "Возможен слабый {}.".format(tp)
    elif p < 0.2:
        rv = "Слабый {}.".format(tp)
    elif p < 0.6: 
        rv = tp[0].upper() + tp[1:] + '.'
    else:
        if t > 3:
            rv = "Ливень"
        elif t < -1 :
            rv = "Снегопад"
        else:
            rv = "Сильный снег с дождем."
    return rv

get_precipitate_phrase(0.5,6)

'Дождь.'

In [139]:
def get_frcst_date_phrase(d):
    
    year_word = words.Word(type=r.WORD_TYPE.NOUN,
                                            forms=['год', 'года', 'году', 'год', 'годом', 'годе', 
                                                   'годов', 'года', 'года', 'годы', 'годами', 'годах', 
                                                   'годов', 'года', 'года', 'годы', 'годами', 'годах'],
                                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))
    
    hour_word = words.Word(type=r.WORD_TYPE.NOUN,
                                            forms=['час', 'часа', 'часу', 'час', 'часом', 'часе', 
                                                   'часы', 'часов', 'часам', 'часы', 'часами', 'часах', 
                                                   'часы', 'часов', 'часам', 'часы', 'часами', 'часах',],
                                            properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))
    
    my_dictionary = dictionary.Dictionary(words=[year_word, hour_word])
    
    dow = []
    dow.append( words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['понедельник', 'понедельника', 'понедельнику', 'понедельник', 'понедельником', 'понедельнике', 
                       'понедельники', 'понедельников', 'понедельникам', 'понедельники', 'понедельниками', 'понедельниках',
                       'понедельники', 'понедельников', 'понедельникам', 'понедельники', 'понедельниками', 'понедельниках'],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))
    dow.append(words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['вторник', 'вторника', 'вторнику', 'вторник', 'вторником', 'вторнике', 
                       'вторники', 'вторников', 'вторникам', 'вторники', 'вторниками', 'вторниках', 
                       'вторники', 'вторников', 'вторникам', 'вторники', 'вторниками', 'вторниках'],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))
    dow.append(words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['среда', 'среды', 'среде', 'среду', 'средой', 'среде', 
                       'среды', 'сред', 'средам', 'среды', 'средами', 'средах',
                       'среды', 'сред', 'средам', 'среды', 'средами', 'средах'],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))
    
    dow.append(words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['четверг', 'четверга', 'четвергу', 'четверг', 'четвергом', 'четверге', 
                       'четверги', 'четвергов', 'четвергам', 'четверги', 'четвергами', 'четвергах',
                       'четверги', 'четвергов', 'четвергам', 'четверги', 'четвергами', 'четвергах'],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))
        
    dow.append(words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['пятница', 'пятницы', 'пятнице', 'пятницу', 'пятницей', 'пятнице',
                       'пятницы', 'пятниц', 'пятницам', 'пятницы', 'пятницами', 'пятницах',
                       'пятницы', 'пятниц', 'пятницам', 'пятницы', 'пятницами', 'пятницах' ],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))
    
    dow.append(words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['суббота', 'субботы', 'субботе', 'субботу', 'субботой', 'субботе', 
                       'субботы', 'суббот', 'субботам', 'субботы', 'субботами', 'субботах', 
                       'субботы', 'суббот', 'субботам', 'субботы', 'субботами', 'субботах'],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))
        
    dow.append(words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['воскресенье', 'воскресения', 'воскресенью', 'воскресенье', 'воскресеньем', 'воскресеньи', 
                       'воскресенья', 'воскресений', 'воскресеньям', 'воскресенья', 'воскресеньями', 'воскресеньях', 
                       'воскресенья', 'воскресений', 'воскресеньям', 'воскресенья', 'воскресеньями', 'воскресеньях'],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))


    months = []
    months.append(words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                forms=['январь', 'января', 'январю', 'январь', 'январём', 'январе', 
                       'январи', 'январей', 'январям', 'январи', 'январями', 'январях', 
                       'январи', 'январей', 'январям', 'январи', 'январями', 'январях'],
                properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.MASCULINE))))
    
    
   
    t1 = templates.Template()
    #template.parse('Сейчас [temp] [градус|temp|вн] [тепло|рд].', externals=('temp'))
    t1.parse('Прогноз погоды на [hour] [час|hour] [dow|рд], [dom] [month|рд] [year] [год|year].', 
                   externals = ('hour','dow', 'dom','month','year'))
    r1 = t1.substitute(externals={
                                'hour': constructors.construct_integer(d.hour),
                                'dow': dow[d.isoweekday() - 1],
                                'dom': constructors.construct_integer(d.day),
                                'month': months[d.month - 1],
                                'year': constructors.construct_integer(d.year)  
                            }, dictionary=my_dictionary)
    return r1

get_frcst_date_phrase(frcst_date)

'Прогноз погоды на 13 часов субботы, 6 января 2018 года.'

# Flood

In [140]:
# Облачность.
# В gfs файлах данные по облачности есть для нескольких уровней:
# низкая, средняя, выская и общая. 
# Код хранится в ключе typeOfFirstFixedSurface, 
# нас интересует вся атмосфера, typeOfFirstFixedSurface = 10
# Коды есть здесь: http://www.cosmo-model.org/content/model/documentation/grib/pdtemplate_4.41.htm , 
# но это неполный список. 
# Более полный удалось найти в исходниках программы zyGrib: http://www.zygrib.org/
clouds = myfile.select(name = 'Total Cloud Cover')
for i in clouds:
    if int(i['typeOfFirstFixedSurface']) == 10:
        moscow_cloud = i.data()[0][MOSCOW_LAT_INDEX][MOSCOW_LON_INDEX]
print("Cloud Cover: {}".format(moscow_cloud))

Cloud Cover: 98.0


In [ ]:
t = myfile.select(name = 'Wind speed (gust)')
t1 = t[0]

In [ ]:
t1.latlons()

In [ ]:
# описываем существительное для словаря
coins_word = words.Word(type=r.WORD_TYPE.NOUN,
                        forms=[ u'монета', u'монеты', u'монете', u'монету', u'монетой', u'монете',    # единственнео число
                                u'монеты', u'монет', u'монетам', u'монеты', u'монетами', u'монетах',  # множественное число
                                u'монеты', u'монет', u'монетам', u'монеты', u'монетами', u'монетах'], # счётное число (заполнено для пример, может быть заполнено методом autofill_missed_forms)
                        properties=words.Properties(r.ANIMALITY.INANIMATE, r.GENDER.FEMININE)) # свойства: неодушевлённое, женский род

# описываем глагол для словаря
action_word = words.Word(type=r.WORD_TYPE.VERB,
                         # описываем только нужны нам формы слова (порядок важен и определён в utg.data.WORDS_CACHES[r.WORD_TYPE.VERB])
                         forms=[u'подарить', u'подарил', u'подарило', u'подарила', u'подарили'] + [u''] * (len(data.WORDS_CACHES[r.WORD_TYPE.VERB]) - 5),
                         properties=words.Properties(r.ASPECT.PERFECTIVE, r.VOICE.DIRECT) )
action_word.autofill_missed_forms() # заполняем пропущенные формы на основе введённых (выбираются наиболее близкие)

# создаём словарь для использования в шаблонах
test_dictionary = dictionary.Dictionary(words=[coins_word, action_word])

# создаём шаблон
template = templates.Template()

# externals — внешние переменные, не обязаны быть в словаре
template.parse(u'[Npc] [подарил|npc] [hero|дт] [coins] [монета|coins|вн].', externals=('hero', 'npc', 'coins'))

# описываем внешние переменные
hero = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                 forms=[u'герой', u'героя', u'герою', u'героя', u'героем', u'герое',
                                        u'герои', u'героев', u'героям', u'героев', u'героями', u'героях',
                                        u'герои', u'героев', u'героям', u'героев', u'героями', u'героях'],
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.MASCULINE)))

npc = words.WordForm(words.Word(type=r.WORD_TYPE.NOUN,
                                forms=[u'русалка', u'русалки', u'русалке', u'русалку', u'русалкой', u'русалке',
                                       u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках',
                                       u'русалки', u'русалок', u'русалкам', u'русалок', u'русалками', u'русалках'],
                                 properties=words.Properties(r.ANIMALITY.ANIMATE, r.GENDER.FEMININE)))

# осуществляем подстановку
result = template.substitute(externals={'hero': hero,
                                        'npc': npc,
                                        'coins': constructors.construct_integer(125)},
                             dictionary=test_dictionary)

result

In [ ]:
int(round(t,0))

In [20]:
def generate_forms(word):
    morph = pymorphy2.MorphAnalyzer()
    p = morph.parse(word)[0]
    res = []
    lexems = p.lexeme
    if 'NOUN' in p.tag:
        for i in range(len(lexems)):
            res.append(lexems[i][4][0][1])
        #print(res)
        if 'femn' in p.tag:
            res1 = res[:5]
            res1.append(res[6])
            res1.extend(res[7:])
            res1.extend(res1[6:])
            return(res1)
        else:
            res.extend(res[6:])
            return(res)
    if 'VERB' in p.tag or 'INFN' in p.tag:
        for i in [0,1,3,2,4]:
            res.append(lexems[i][4][0][1])
        return(res)
    if 'ADJF' in p.tag:
        for i in range(18):
            res.append(lexems[i][4][0][1])
        return(res)

In [127]:
print(generate_forms('воскресенье'))

['воскресение', 'воскресенье', 'воскресения', 'воскресенья', 'воскресению', 'воскресенью', 'воскресение', 'воскресенье', 'воскресением', 'воскресеньем', 'воскресении', 'воскресенье', 'воскресеньи', 'воскресения', 'воскресенья', 'воскресений', 'воскресениям', 'воскресеньям', 'воскресения', 'воскресенья', 'воскресениями', 'воскресеньями', 'воскресениях', 'воскресеньях', 'воскресение', 'воскресенье', 'воскресением', 'воскресеньем', 'воскресении', 'воскресенье', 'воскресеньи', 'воскресения', 'воскресенья', 'воскресений', 'воскресениям', 'воскресеньям', 'воскресения', 'воскресенья', 'воскресениями', 'воскресеньями', 'воскресениях', 'воскресеньях']
